In [1]:
# noexport

import os
os.system('export_notebook identify_domain_v3.ipynb')

0

In [2]:
from tmilib import *

In [3]:
#user = get_test_users()[0]
#print user

In [4]:
#tab_focus_times = get_tab_focus_times_for_user(user)
#print tab_focus_times[0].keys()

In [5]:
#print active_second_to_domain_id.keys()[:10]

In [6]:
#seen_domains_only_real = get_seen_urls_and_domains_real_and_history()['seen_domains_only_real']

In [7]:
#[domain_to_id(x) for x in ['www.mturk.com', 'apps.facebook.com', 'www.facebook.com', 'www.reddit.com', 'www.youtube.com']]

In [8]:
#sumkeys(total_tabbed_back_domains_all, 'www.mturk.com', 'apps.facebook.com', 'www.facebook.com', 'www.reddit.com', 'www.youtube.com')

In [9]:
#728458.0/2382221

In [10]:
#print_counter(total_tabbed_back_domains_first)

In [11]:
'''
total_active_seconds = 0

for user in get_test_users():
  total_active_seconds += len(get_active_seconds_for_user(user))
  #for span in get_active_seconds_for_user(user)
print total_active_seconds
'''

'\ntotal_active_seconds = 0\n\nfor user in get_test_users():\n  total_active_seconds += len(get_active_seconds_for_user(user))\n  #for span in get_active_seconds_for_user(user)\nprint total_active_seconds\n'

In [12]:
#for user in get_test_users():
#  print user
#  print (get_recently_seen_domain_stats_for_user(user)['stats'])

In [19]:
@memoized
def get_recently_seen_domain_stats_for_user_v3(user):
  ordered_visits = get_history_ordered_visits_corrected_for_user(user)
  ordered_visits = exclude_bad_visits(ordered_visits)
  active_domain_at_time = get_active_domain_at_time_for_user(user)
  active_seconds_set = set(get_active_insession_seconds_for_user(user))
  #active_second_to_domain_id = get_active_second_to_domain_id_for_user(user)
  active_second_to_domain_id = {int(k):v for k,v in get_active_second_to_domain_id_for_user(user).viewitems()}
  recently_seen_domain_ids = [-1]*100
  seen_domain_ids_set = set()
  stats = Counter()
  nomatch_domains = Counter()
  tabbed_back_domains_first = Counter()
  tabbed_back_domains_second = Counter()
  tabbed_back_domains_all = Counter()
  distractors_list = [domain_to_id(x) for x in ['www.mturk.com', 'apps.facebook.com', 'www.facebook.com', 'www.reddit.com', 'www.youtube.com']]
  distractors = set(distractors_list)
  most_recent_distractor = distractors_list[0]
  for idx,visit in enumerate(ordered_visits):
    if idx+1 >= len(ordered_visits):
      break
    next_visit = ordered_visits[idx+1]
    cur_domain = url_to_domain(visit['url'])
    cur_domain_id = domain_to_id(cur_domain)
    if cur_domain_id in distractors:
      most_recent_distractor = cur_domain_id
    recently_seen_domain_ids.append(cur_domain_id)
    #if cur_domain_id != recently_seen_domain_ids[-1]:
    #  if cur_domain_id in seen_domain_ids_set:
    #    recently_seen_domain_ids.remove(cur_domain_id)
    #  seen_domain_ids_set.add(cur_domain_id)
    #  recently_seen_domain_ids.append(cur_domain_id)
    #if cur_domain_id != recently_seen_domain_ids[-1]:
    #  if cur_domain_id in seen_domain_ids_set:
    #    recently_seen_domain_ids.remove(cur_domain_id)
    #  seen_domain_ids_set.add(cur_domain_id)
    #  recently_seen_domain_ids.append(cur_domain_id)
    next_domain = url_to_domain(next_visit['url'])
    next_domain_id = domain_to_id(next_domain)
    cur_time_sec = int(round(visit['visitTime'] / 1000.0))
    next_time_sec = int(round(next_visit['visitTime'] / 1000.0))
    
    if cur_time_sec > next_time_sec:
      continue
    
    for time_sec in xrange(cur_time_sec, next_time_sec+1):
      if time_sec not in active_seconds_set:
        continue
      ref_domain_id = active_second_to_domain_id[time_sec]
      stats['total'] += 1
      if most_recent_distractor == ref_domain_id:
        stats['most_recent_distractor_total'] += 1
        if cur_domain_id == ref_domain_id:
          stats['most_recent_distractor_curdomain'] += 1
        elif cur_domain_id == next_domain_id:
          stats['most_recent_distractor_nextdomain'] += 1
        else:
          stats['most_recent_distractor_some_prev_domain'] += 1
      if cur_domain_id == ref_domain_id:
        if next_domain_id == cur_domain_id:
          stats['first and next equal and correct'] += 1
        else:
          stats['first correct only'] += 1
        continue
      if next_domain_id == ref_domain_id:
        stats['next correct only'] += 1
        continue
      stats['both incorrect'] += 1
      found_match = False
      ref_domain = id_to_domain(ref_domain_id)
      for i in range(1,101):
        if recently_seen_domain_ids[-1-i] == ref_domain_id:
          stats['nth previous correct ' + str(abs(i))] += 1
          stats['some previous among past 100 correct'] += 1
          found_match = True
          tabbed_back_domains_all[ref_domain] += 1
          if i == 1:
            tabbed_back_domains_first[ref_domain] += 1
          if i == 2:
            tabbed_back_domains_second[ref_domain] += 1
          break
      if not found_match:
        ref_domain = id_to_domain(ref_domain_id)
        #if ref_domain == 'newtab':
        #  stats['newtab'] += 1
        #  continue
        stats['no match found'] += 1
        nomatch_domains[id_to_domain(ref_domain_id)] += 1
      '''
      if cur_domain_id == ref_domain_id:
        if next_domain_id == cur_domain_id:
          stats['first and next equal and correct'] += 1
          continue
        else:
          stats['first correct only'] += 1
          continue
      else:
        if next_domain_id == cur_domain_id:
          stats['both incorrect'] += 1
          found_match = False
          for i in range(1,101):
            if recently_seen_domain_ids[-1-i] == ref_domain_id:
              stats['nth previous correct ' + str(abs(i))] += 1
              stats['some previous among past 100 correct'] += 1
              found_match = True
              break
          if not found_match:
            ref_domain = id_to_domain(ref_domain_id)
            if ref_domain == 'newtab':
              stats['newtab'] += 1
              continue
            stats['no match found'] += 1
            nomatch_domains[id_to_domain(ref_domain_id)] += 1
          continue
        if next_domain_id == ref_domain_id:
          stats['next correct only'] += 1
          continue
      '''
  return {
    'stats': stats,
    'nomatch_domains': nomatch_domains,
    'tabbed_back_domains_all': tabbed_back_domains_all,
    'tabbed_back_domains_first': tabbed_back_domains_first,
    'tabbed_back_domains_second': tabbed_back_domains_second,
  }

In [20]:
#total_stats = Counter({'total': 544544, 'first and next equal and correct': 351081, 'first correct only': 88522, 'both incorrect': 51663, 'some previous among past 20 correct': 41231, 'nth previous correct 1': 31202, 'next correct only': 23569, 'no match found': 10432, 'nth previous correct 2': 3311, 'nth previous correct 3': 1635, 'nth previous correct 4': 905, 'nth previous correct 5': 862, 'nth previous correct 6': 545, 'nth previous correct 7': 412, 'nth previous correct 8': 357, 'nth previous correct 9': 269, 'nth previous correct 10': 259, 'nth previous correct 13': 234, 'nth previous correct 11': 229, 'nth previous correct 12': 190, 'nth previous correct 15': 183, 'nth previous correct 14': 140, 'nth previous correct 17': 139, 'nth previous correct 20': 95, 'nth previous correct 16': 90, 'nth previous correct 19': 88, 'nth previous correct 18': 86})

total_stats = Counter()

for user in get_test_users():
  for k,v in get_recently_seen_domain_stats_for_user_v3(user)['stats'].viewitems():
    total_stats[k] += v
#total_stats = Counter({'total': 544544, 'first and next equal and correct': 351081, 'first correct only': 88522, 'both incorrect': 51663, 'some previous among past 20 correct': 41136, 'nth previous correct 2': 31202, 'next correct only': 23569, 'no match found': 10527, 'nth previous correct 3': 3311, 'nth previous correct 4': 1635, 'nth previous correct 5': 905, 'nth previous correct 6': 862, 'nth previous correct 7': 545, 'nth previous correct 8': 412, 'nth previous correct 9': 357, 'nth previous correct 10': 269, 'nth previous correct 11': 259, 'nth previous correct 14': 234, 'nth previous correct 12': 229, 'nth previous correct 13': 190, 'nth previous correct 16': 183, 'nth previous correct 15': 140, 'nth previous correct 18': 139, 'nth previous correct 17': 90, 'nth previous correct 20': 88, 'nth previous correct 19': 86})

In [21]:
print_counter(total_stats)

total 14806666
first and next equal and correct 6588587
most_recent_distractor_total 4978110
first correct only 4430669
most_recent_distractor_curdomain 4226119
both incorrect 2692997
some previous among past 100 correct 2366817
next correct only 1094413
most_recent_distractor_some_prev_domain 621347
nth previous correct 1 547695
no match found 326180
nth previous correct 2 298404
nth previous correct 3 196462
nth previous correct 4 143826
most_recent_distractor_nextdomain 130644
nth previous correct 5 109180
nth previous correct 6 91930
nth previous correct 7 83033
nth previous correct 8 63701
nth previous correct 9 59038
nth previous correct 10 47744
nth previous correct 11 47014
nth previous correct 12 39476
nth previous correct 13 36503
nth previous correct 14 31132
nth previous correct 15 30979
nth previous correct 16 27317
nth previous correct 17 24380
nth previous correct 20 23592
nth previous correct 19 23271
nth previous correct 18 22857
nth previous correct 22 17208
nth previ

In [15]:
print_counter(total_stats)

total 14806666
first and next equal and correct 6588587
most_recent_distractor_total 4978110
first correct only 4430669
most_recent_distractor_curdomain 4226119
both incorrect 3787410
some previous among past 100 correct 3418250
nth previous correct 1 854163
most_recent_distractor_some_prev_domain 621347
nth previous correct 2 465501
no match found 369160
nth previous correct 3 287350
nth previous correct 4 216630
nth previous correct 5 160815
nth previous correct 6 137290
most_recent_distractor_nextdomain 130644
nth previous correct 7 110543
nth previous correct 8 90950
nth previous correct 9 78637
nth previous correct 10 66622
nth previous correct 11 59158
nth previous correct 12 56612
nth previous correct 13 47655
nth previous correct 14 42790
nth previous correct 15 40318
nth previous correct 16 39216
nth previous correct 18 31599
nth previous correct 20 30248
nth previous correct 17 30108
nth previous correct 19 30054
nth previous correct 21 25372
nth previous correct 22 22374
nth

In [16]:
def sumkeys(d, *args):
  return sum(d.get(x, 0.0) for x in args)

In [22]:
norm = {k:float(v)/total_stats['total'] for k,v in total_stats.viewitems()}
print 'select prev gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only')
print 'prev or next gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only')
print 'prev or next or newtab gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only', 'newtab')
for i in range(1, 101):
  sumprev = sum([norm.get('nth previous correct '+str(x),0.0) for x in range(i+1)])
  print 'prev or next or past ' + str(i), sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only', 'newtab')+sumprev

select prev gets answer correct 0.74420912851
prev or next gets answer correct 0.818122661779
prev or next or newtab gets answer correct 0.818122661779
prev or next or past 1 0.855112420311
prev or next or past 2 0.875265775564
prev or next or past 3 0.888534258826
prev or next or past 4 0.898247856742
prev or next or past 5 0.905621562612
prev or next or past 6 0.911830252671
prev or next or past 7 0.91743806472
prev or next or past 8 0.921740248615
prev or next or past 9 0.925727506786
prev or next or past 10 0.928952000403
prev or next or past 11 0.932127191901
prev or next or past 12 0.934793288374
prev or next or past 13 0.937258596905
prev or next or past 14 0.939361163411
prev or next or past 15 0.941453396734
prev or next or past 16 0.943298309018
prev or next or past 17 0.944944864698
prev or next or past 18 0.946488561301
prev or next or past 19 0.948060218283
prev or next or past 20 0.949653554689
prev or next or past 21 0.950705851
prev or next or past 22 0.951868030251
pre

In [17]:
norm = {k:float(v)/total_stats['total'] for k,v in total_stats.viewitems()}
print 'select prev gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only')
print 'prev or next gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only')
print 'prev or next or newtab gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only', 'newtab')
for i in range(1, 101):
  sumprev = sum([norm.get('nth previous correct '+str(x),0.0) for x in range(i+1)])
  print 'prev or next or past ' + str(i), sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only', 'newtab')+sumprev

select prev gets answer correct 0.74420912851
prev or next gets answer correct 0.74420912851
prev or next or newtab gets answer correct 0.74420912851
prev or next or past 1 0.801896861859
prev or next or past 2 0.83333547201
prev or next or past 3 0.852742271623
prev or next or past 4 0.867372844096
prev or next or past 5 0.878233830627
prev or next or past 6 0.887506005741
prev or next or past 7 0.894971764744
prev or next or past 8 0.901114268398
prev or next or past 9 0.906425187142
prev or next or past 10 0.910924647047
prev or next or past 11 0.914920009677
prev or next or past 12 0.918743422726
prev or next or past 13 0.921961905536
prev or next or past 14 0.924851820119
prev or next or past 15 0.927574782871
prev or next or past 16 0.930223319686
prev or next or past 17 0.932256728152
prev or next or past 18 0.934390834507
prev or next or past 19 0.936420595967
prev or next or past 20 0.938463459634
prev or next or past 21 0.940177012165
prev or next or past 22 0.941688088325
pr

In [18]:
#print norm['most_recent_distractor_total']
#print norm['most_recent_distractor_curdomain']
#print norm['most_recent_distractor_nextdomain']
#print norm['most_recent_distractor_some_prev_domain']